In [ ]:
import numpy as np
import matplotlib

matplotlib.use("agg")

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


class HistogramNormalize(matplotlib.colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None, mixing_degree=1):
        self.mixing_degree = mixing_degree
        if vmin is not None:
            data = data[data >= vmin]
        if vmax is not None:
            data = data[data <= vmax]

        self.sorted_data = np.sort(data.flatten())
        matplotlib.colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        hist_norm = np.ma.masked_array(
            np.searchsorted(self.sorted_data, value) / len(self.sorted_data)
        )
        linear_norm = super().__call__(value, clip)
        return self.mixing_degree * hist_norm + (1 - self.mixing_degree) * linear_norm


golden_mean = (np.sqrt(5) - 1) / 2  # Aesthetic ratio
fig_width_pt = 246.0  # Columnwidth
inches_per_pt = 1 / 72.27  # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean  # height in inches
fig_size = [fig_width, fig_height]

mpl_params = {
    "backend": "ps",
    "axes.labelsize": 13,
    "font.size": 13,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "text.usetex": True,
    "figure.figsize": fig_size,
    "font.family": "serif",
    "font.serif": "Computer Modern Roman",
    "legend.frameon": True,
    "savefig.dpi": 300,
}

plt.rcParams.update(mpl_params)
plt.rc("text.latex", preamble=[r"\usepackage{xfrac}", r"\usepackage{siunitx}"])


def gist_heat_r_transparent():
    import matplotlib.cm
    import matplotlib.colors as mcolors

    colors = matplotlib.cm.gist_heat_r(np.linspace(0, 1, 128))
    colors[:, 3] = np.linspace(0, 1, 128).tolist()
    gist_heat_r_transparent = mcolors.LinearSegmentedColormap.from_list(
        "gist_heat_r_transparent", colors
    )
    return gist_heat_r_transparent


gray_r = matplotlib.cm.gray_r
gray_r.set_bad("#e0e0e0")


def num_to_latex_exp(x, only_exp=True, N_digits=1):
    if not only_exp:
        if N_digits == 1:
            num, exponent = f"{x:.0e}".split("e")
        elif N_digits == 2:
            num, exponent = f"{x:.1e}".split("e")
        return rf"{num} \times 10^{{{int(exponent)}}}"
    else:
        exponent = np.floor(np.log10(np.abs(x))).astype(int)
        return f"10^{{{int(exponent)}}}"
    

def color_spline(ax, c):
    ax.spines["bottom"].set_color(c)
    ax.spines["top"].set_color(c)
    ax.spines["right"].set_color(c)
    ax.spines["left"].set_color(c)
    ax.tick_params(axis="x", colors=c)
    ax.tick_params(axis="y", colors=c)
    for tick in ax.get_yticklabels():
        tick.set_color("k")
    for tick in ax.get_xticklabels():
        tick.set_color("k")

In [ ]:
from learners_file import *

In [ ]:
adaptive.notebook_extension()

In [ ]:
learner = learners[-1]

lead_pars = learner.function.keywords["lead_pars"]
params = learner.function.keywords["params"]
lead = phase_diagram.make_lead(**dict(lead_pars, wraparound=True)).finalized()
B_xs = np.linspace(*learner.bounds[0], 100)
phase_bounds = []
for B_x in B_xs:
    params["B_x"] = B_x
    phase_bounds.append(
        phase_diagram.find_phase_bounds(
            lead, phase_diagram.parse_params(params), num_bands=50, mu_param="mu_lead"
        )[::2]
    )

data = learner.plot(n=200).Image.I.data

In [ ]:
def bands(k_x, mu, B_x, params=params, lead=lead):
    H = lead.hamiltonian_submatrix(params=dict(params, B_x=B_x, k_x=k_x, mu_lead=mu))
    return np.linalg.eigvalsh(H)

mu_mid = phase_bounds[len(B_xs)//2][0]  # first_phase_bound
B_x = B_xs[len(B_xs)//2]
eps = 1
mu_before = mu_mid - eps
mu_after = mu_mid + eps
ks = np.linspace(-0.5, 0.5, 301)
E_mid = np.array([bands(k, mu=mu_mid, B_x=B_x) for k in ks])
E_before = np.array([bands(k, mu=mu_before, B_x=B_x) for k in ks])
E_after = np.array([bands(k, mu=mu_after, B_x=B_x) for k in ks])


In [ ]:
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt

fig, axs = plt.subplots(
    2, 2, sharex=False, sharey=False, figsize=(2 * fig_width, 3 * fig_height)
)
plt.subplots_adjust(
    bottom=0.2, left=0.125, right=0.85, top=0.8, hspace=0.5, wspace=0.32
)
(ax1, ax2), (ax3, ax4) = axs


def plot_learner_on_ax(ax, learner):
    im = learner.plot(n=400).Image.I
    l, b, r, t = im.bounds.lbrt()
    data = im.data
    data[data < 0] = data[data < 0] * -1
    return ax.imshow(
        im.data,
        extent=(l, r, b, t),
        aspect="auto",
        cmap="viridis",
        norm=HistogramNormalize(data, vmin=0, vmax=0.5),
    )


for c, mu in zip(["C2", "C3", "C4"], [mu_before, mu_mid, mu_after]):
    ax1.scatter([B_x], [mu], edgecolors=c, zorder=4, facecolors="none")

color_spline(ax2, "C2")
color_spline(ax3, "C3")
color_spline(ax4, "C4")
ax2.plot(ks, E_before, c="k")
ax3.plot(ks, E_mid, c="k")
ax4.plot(ks, E_after, c="k")

for ax in [ax2, ax3, ax4]:
    ax.set_ylim(-2.3, 2.3)
    ax.set_xlabel("$a k_x$")
    ax.set_ylabel("$E$ (meV)")
    xvals = [-np.pi / 4, 0, np.pi / 4]
    xlabels = ["$-\pi/4$", r"$0$", "$\pi/4$"]
    ax.set_xticks(xvals)
    ax.set_xticklabels(xlabels)


ax1.plot(B_xs, phase_bounds, c="C6", lw=1, ls="--")
im = plot_learner_on_ax(ax1, learner)
ax1.set_ylim(0, 15)
ax1.set_xlabel("$B_x$ (T)")
ax1.set_ylabel("$\mu$ (meV)")

for i, ax in enumerate(axs.reshape(-1)):
    label = "abcdef"[i]
    ax.text(
        0.995,
        0.97,
        f"$\mathrm{{({label})}}$",
        transform=ax.transAxes,
        verticalalignment="top",
        horizontalalignment="right",
        fontsize=14,
        color="black" if i >= 1 else "white",
    )
    if i == 0:
        ax.set_title("Phase diagram")
    else:
        mu_text = r"$\mu={:.0f}$ meV".format([mu_before, mu_mid, mu_after][i - 1])
        ax.set_title(mu_text)

cb = fig.colorbar(im, ax=ax1, extend="max")
cbar_ticks = [0, 0.5]
cb.set_ticks(cbar_ticks)
cb.set_label(r"$E_{\mathrm{gap}}$ (meV)", labelpad=-13)

plt.savefig("phase_diagram_with_bands.pdf", bbox_inches="tight", transparent=True)
plt.show()

# generalized ev problem

In [ ]:
learner = learners[-1]
lead_pars = learner.function.keywords["lead_pars"]
params = learner.function.keywords["params"]
params = dict(phase_diagram.parse_params(params), B_x=B_x)
lead = phase_diagram.make_lead(**dict(lead_pars, wraparound=True)).finalized()
phase_bounds = phase_diagram.find_phase_bounds(
    lead, params, num_bands=50, mu_param="mu_lead"
)[::2]
mus = np.linspace(0, 25, 201)
Es = []
for params["mu_lead"] in mus:
    H = lead.hamiltonian_submatrix(params=dict(params, k_x=0))
    Es.append(np.linalg.eigvalsh(H))
Es = np.array(Es)

In [ ]:
fig, ax = plt.subplots(figsize=(fig_width, 2 * fig_height))
ax.plot(mus, Es, c="k", lw=0.5)
y_min_max = (-15, 15)
ax.set_xlim(mus[0], mus[-1])
ax.set_ylim(y_min_max)
ax.hlines(0, mus[0], mus[-1], color="C1", alpha=0.3)
ax.scatter(
    phase_bounds,
    np.zeros_like(phase_bounds),
    edgecolors="C1",
    alpha=1,
    zorder=10,
    facecolors="none",
)

mu_k = 2
i = np.abs(mus - mu_k).argmin()

ax.vlines(mu_k, *y_min_max, color="C2", alpha=0.3)
ax.scatter(
    mu_k * np.ones_like(Es[i]),
    Es[i],
    edgecolors="C2",
    alpha=1,
    zorder=10,
    facecolors="none",
)
ax.set_xlabel("$\mu$ (meV)")
ax.set_ylabel("$E(k=0)$ (meV)")

# To specify the number of ticks on both or any single axes
plt.locator_params(axis='y', nbins=4)
plt.locator_params(axis='x', nbins=4)
plt.savefig("figures/generalized_ev_problem.pdf", bbox_inches="tight", transparent=True)
